In [ ]:
# ! pip install tensorflow-addons
# ! pip install tensorflow==2.3.1
! pip list | grep tensorflow

In [ ]:
import json
from pathlib import Path
import os
import pickle
import random
import shutil
import sys
import numpy as np

import glob2 as glob
import tensorflow as tf
from tensorflow.keras import callbacks, layers, models, Model
import matplotlib.pyplot as plt
%matplotlib inline
from azureml.core import Dataset, Workspace
from azureml.core.run import Run

sys.path.append(str(Path(os.getcwd()).parents[2]))  # src/ dir
from common.model_utils.utils import get_dataset_path, download_dataset

from data_utils.data_visualisation.visualization_utils import (IMAGE_TARGET_HEIGHT, 
    IMAGE_TARGET_WIDTH, REPO_DIR, path_to_ndarray, choose_a_pickle_file, get_datetime)

from common.model_utils.preprocessing import preprocess_depthmap

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
dataset_name = "anon-realtime-testdata"  # "anon-realtime-evaluation"
DEPTHMAPS_DIR_NAME = "scans" #  "depthmaps"
sample_qr_code = "1585348358-4ek1lj215y"

In [ ]:
workspace = Workspace.from_config()
dataset = Dataset.get_by_name(workspace, name=dataset_name)
workspace, dataset

In [ ]:
# mount_context = dataset.mount()
# mount_context.start()  # this will mount the file streams
# mount_context.mount_point
# ! ls {mount_context.mount_point}
# ! ls {mount_context.mount_point + '/depthmaps'}

In [ ]:
# Downlaod dataset (prefer over mounting !)
DATA_DIR_ONLINE_RUN = Path("/tmp/data/")
dataset_path = get_dataset_path(DATA_DIR_ONLINE_RUN, dataset_name)
download_dataset(workspace, dataset_name, dataset_path)

In [ ]:
depthmaps_paths = Path(f"/tmp/data/{dataset_name}/{DEPTHMAPS_DIR_NAME}")

In [ ]:
qrcode_path_ = depthmaps_paths / sample_qr_code
qrcode_path = str(qrcode_path_ / "100/")  # contains .p files

assert os.path.exists(qrcode_path)

path_with_wildcard = os.path.join(qrcode_path, "*.p")
list_of_pickle_file_paths = sorted(glob.glob(path_with_wildcard))

In [ ]:
list_of_pickle_file_paths

In [ ]:
# TARGET contain:  height, weight, muac, age in days, gender('male' or 'female'), quality ('good' or 'bad'), 'test'
TARGET_INDEXES = [0, 3, ]  # height, age in days  

IMAGE_TARGET_HEIGHT = 240
IMAGE_TARGET_WIDTH = 180

def show_pickle(pickle_file_path):
    depthmap, _targets = py_load_pickle(pickle_file_path)
    depthmap = tf.reshape(depthmap, (IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH))
    plt.imshow(depthmap.numpy(), cmap='gray', vmin=0, vmax=1)
    print(f"height: {_targets[0]:.2f} cm")
    print(f"age: {_targets[1]:.2f} days")

def preprocess_targets(targets, targets_indices):
    if targets_indices is not None:
        targets = targets[targets_indices]
    return targets.astype("float32")

def py_load_pickle(path, max_value=7.5):
    path_ = path if isinstance(path, str) else path.numpy()
    depthmap, targets = pickle.load(open(path_, "rb"))
    depthmap = preprocess_depthmap(depthmap)
    depthmap = depthmap / max_value
    depthmap = tf.image.resize(depthmap, (IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH))
    targets = preprocess_targets(targets, TARGET_INDEXES)
    return depthmap, targets

In [ ]:
show_pickle(list_of_pickle_file_paths[5])

In [ ]:
depthmap.shape, _targets

In [ ]:
# Setup prediction
service_name = 'q3-depthmap-height-run-01-ci'
service = workspace.webservices[service_name]

# In evaluate.py:
def get_height_prediction(model_path, depthmap_path_list):
    
    # Choose one artifact and predict on it  # TODO change to multiple
    pickle_file_path = depthmap_path_list[5]

    # Path to numpy array
    depthmap, _targets = py_load_pickle(pickle_file_path)
    depthmap_np = depthmap.numpy()
    depthmaps = np.array([depthmap_np]).reshape(1, 240, 180, 1)
    depthmaps_json = json.dumps({'data': depthmaps.tolist()})

    # Predict
    predictions = service.run(input_data=depthmaps_json)
    return predictions

In [ ]:
get_height_prediction(None, list_of_pickle_file_paths)

In [ ]:
print(f"prediction is: {predictions[0][0]} \n" + 
      f"scan label is: {_targets[0]}"
)

In [ ]:
# Maybe load model here into memory